In [9]:
import networkx as nx
import random


def gen_erdos_renyi(n, p, K):
    graphs = []
    for _ in range(K):
        # Generate a directed Erdős-Rényi graph G(n, p)
        G = nx.gnp_random_graph(n, p, directed=True)

        # Assign random capacities and add reverse edges with the same capacity
        for u, v in list(G.edges()):
            capacity = random.uniform(0, 1)
            G[u][v]["capacity"] = capacity
            G.add_edge(v, u, capacity=capacity)

        graphs.append(G)
    return graphs

# n = 1000
# p = 0.1
# K = 50
# generated_graphs = gen_erdos_renyi(n, p, K)

# for i, g in enumerate(generated_graphs, start=1):
#     print(f"Graph {i}:")
#     print("Nodes:", list(g.node_indexes()))
#     print("Edges with weights:")
#     for (u, v, wt) in g.weighted_edge_list():
#         print(f"  {u} -> {v} (weight={wt})")


In [10]:
import heapq


def gen_random_trees(n, K):
    def prufer_to_tree(prufer):
        degree = [1] * n
        edges = []

        # Increase degree for each node in prufer
        for x in prufer:
            degree[x] += 1

        # Leaves are nodes with degree=1
        leaves = [i for i, d in enumerate(degree) if d == 1]
        heapq.heapify(leaves)

        # Decode prufer: repeatedly connect smallest leaf to next code element
        for x in prufer:
            leaf = heapq.heappop(leaves)
            edges.append((leaf, x))
            degree[x] -= 1
            if degree[x] == 1:
                heapq.heappush(leaves, x)

        # Connect the last two leaves
        edges.append((heapq.heappop(leaves), heapq.heappop(leaves)))

        return edges

    graphs = []
    for _ in range(K):
        # Generate a random prufer code and build the graph
        prufer = [random.randrange(n) for _ in range(n - 2)]
        edges = prufer_to_tree(prufer)

        G = nx.DiGraph()
        G.add_nodes_from(range(n))

        # For each undirected edge (u, v), add both (u, v) and (v, u) with the same capacity
        for (u, v) in edges:
            capacity = random.random()
            G.add_edge(u, v, capacity=capacity)
            G.add_edge(v, u, capacity=capacity)

        # Add to graphs
        graphs.append(G)

    return graphs

In [11]:
from ortools.linear_solver import pywraplp


def max_flow_glop(G, s, t):
    # Get list of node indices
    nodes = list(G.nodes())

    # Create the LP solver
    solver = pywraplp.Solver.CreateSolver('GLOP')

    # Create variables for flows on edges
    flow = {}
    in_edges = {node: [] for node in nodes}
    out_edges = {node: [] for node in nodes}

    # Iterate over edges to create flow variables
    for u, v, data in G.edges(data=True):
        cap = data['capacity']
        flow[(u, v)] = solver.NumVar(0, cap, f'x_{u}_{v}')
        out_edges[u].append((u, v))
        in_edges[v].append((u, v))

    # Flow conservation constraints for all nodes except s and t
    for node in nodes:
        if node == s or node == t:
            continue
        node_in = solver.Sum([flow[(u, node)] for (u, node) in in_edges[node]])
        node_out = solver.Sum([flow[(node, v)] for (node, v) in out_edges[node]])
        solver.Add(node_in == node_out)

    # Objective: Maximize total flow into sink t
    t_in = solver.Sum([flow[(u, t)] for (u, t) in in_edges[t]])
    t_out = solver.Sum([flow[(t, v)] for (t, v) in out_edges[t]])
    net_flow = t_in - t_out
    solver.Maximize(net_flow)

    # Solve the LP problem
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        max_flow_value = net_flow.solution_value()
        print(f"Maximum flow from {G[s]} to {G[t]}: {max_flow_value}")

        # Optionally, print the flow values on the edges
        for (u, v), var in flow.items():
            flow_value = var.solution_value()
            if flow_value > 0:
                print(f"Flow from {u} to {v}: {flow_value}")
    else:
        print("The solver did not find an optimal solution.")

In [12]:
def max_flow_dinitz(G, s, t):
    from networkx.algorithms.flow import dinitz
    R = dinitz(G, s, t)
    return R.graph['flow_value']


def max_flow_edmonds_karp(G, s, t):
    from networkx.algorithms.flow import edmonds_karp
    R = edmonds_karp(G, s, t, capacity='capacity')
    return R.graph['flow_value']


def max_flow_preflow_push(G, s, t):
    from networkx.algorithms.flow import preflow_push
    R = preflow_push(G, s, t, capacity='capacity')
    return R.graph['flow_value']

In [13]:
G = nx.DiGraph()
G.add_nodes_from(list(range(8)))

edges = [
    (0, 1, {"capacity": 1}),
    (0, 4, {"capacity": 1}),
    (1, 2, {"capacity": 1}),
    (2, 3, {"capacity": 1}),
    (3, 7, {"capacity": 1}),
    (4, 3, {"capacity": 1}),
    (4, 5, {"capacity": 1}),
    (5, 6, {"capacity": 1}),
    (6, 7, {"capacity": 1}),
]

G.add_edges_from(edges)
max_flow_glop(G, 0, 7)

print(max_flow_dinitz(G, 0, 7))
print(max_flow_edmonds_karp(G, 0, 7))
print(max_flow_preflow_push(G, 0, 7))

Maximum flow from {1: {'capacity': 1}, 4: {'capacity': 1}} to {}: 2.0
Flow from 0 to 1: 1.0
Flow from 0 to 4: 1.0
Flow from 1 to 2: 1.0
Flow from 2 to 3: 1.0
Flow from 3 to 7: 1.0
Flow from 4 to 5: 1.0
Flow from 5 to 6: 1.0
Flow from 6 to 7: 1.0
2
2
2


In [15]:
methods = [
    (max_flow_dinitz, "Dinitz"),
    (max_flow_edmonds_karp, "Edmonds-Karp"),
    (max_flow_preflow_push, "Preflow-Push"),
]

In [19]:
import random


def sample_node_pairs(G, S):
    pairs = []
    for _ in range(S):
        u, v = random.sample(G.nodes(), 2)
        pairs.append((u, v))
    return pairs


In [ ]:
import time
import matplotlib.pyplot as plt


def run_erdos_renyi(methods, n_vals, K=30, S=10):
    # Initialize a dictionary to store average runtimes per method across all n_vals
    average_runtimes = {method_name: [] for _, method_name in methods}

    # Loop over each graph size
    for n in n_vals:
        # Build K random graphs of size n
        p = 0.5
        graphs = gen_erdos_renyi(n, p, K)

        # Initialize a dictionary to store runtimes per method at the current graph size
        runtimes_at_current_size = {method_name: [] for _, method_name in methods}

        # Loop over each graph
        for G in graphs:
            # Sample S source-sink pairs from G
            source_sink_pairs = sample_node_pairs(G, S)

            # Dictionary to accumulate runtimes per method within the current graph
            runtimes_in_current_graph = {method_name: [] for _, method_name in methods}

            # Loop over each source-sink pair
            for s, t in source_sink_pairs:
                # Loop over each method
                for method, method_name in methods:
                    # Time the method's solve function
                    start_time = time.time()
                    method(G, s, t)
                    end_time = time.time()

                    # Calculate elapsed time and store it
                    elapsed_time = end_time - start_time
                    runtimes_in_current_graph[method_name].append(elapsed_time)

            # Compute average runtime over all pairs for each method in the current graph
            for method_name in runtimes_in_current_graph:
                avg_runtime = sum(runtimes_in_current_graph[method_name]) / len(runtimes_in_current_graph[method_name])
                runtimes_at_current_size[method_name].append(avg_runtime)

        # Compute average runtime over all graphs for each method at the current graph size
        for method_name in runtimes_at_current_size:
            avg_runtime_over_graphs = sum(runtimes_at_current_size[method_name]) / len(
                runtimes_at_current_size[method_name])
            average_runtimes[method_name].append(avg_runtime_over_graphs)

    # Plotting the results
    plt.figure(figsize=(10, 6))
    for method_name in average_runtimes:
        plt.plot(n_vals, average_runtimes[method_name], marker='o', label=method_name)

    plt.xlabel('Number of Nodes (n)')
    plt.ylabel('Average Runtime (seconds)')
    plt.title('Average Max-Flow Computation Time vs. Graph Size')
    plt.legend()
    plt.grid(True)
    plt.show()
